In [ ]:
import pandas as pd  
import numpy as np  
from sklearn.ensemble import RandomForestRegressor ### 使用 RandomForestClassifier 填补缺失的年龄属性 
from pandas import Series,DataFrame 
import sklearn.preprocessing as preprocessing 
from sklearn import linear_model # 取出我们要的属性值 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
data_train = pd.read_csv("/Users/sunhao/Desktop/Train.csv") 

###通过分析题中的数据，
def set_missing_ages(df): # 把已有的数值型特征取出来丢进Random Forest Regressor中 
    age_df = df[['Age','Fare', 'Parch', 'SibSp', 'Pclass']] # 乘客分成已知年龄和未知年龄两部分 
    known_age = age_df[age_df.Age.notnull()].as_matrix() 
    unknown_age = age_df[age_df.Age.isnull()].as_matrix()
    # y即目标年龄 
    y = known_age[:, 0] 
    # X即特征属性值 
    X = known_age[:, 1:] 
    # fit到RandomForestRegressor之中 
    rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1) 
    rfr.fit(X, y) 
    # 用得到的模型进行未知年龄结果预测 
    predictedAges = rfr.predict(unknown_age[:, 1::]) 
    # 用得到的预测结果填补原缺失数据 
    df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 
    return df, rfr 

data_train, rfr = set_missing_ages(data_train) 
dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked') 
dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex') 
dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass') 
df = pd.concat([data_train,  dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1) 
df.drop(['Pclass', 'Name', 'Sex', 'Ticket',  'Embarked'], axis=1, inplace=True)#将Cabin特征去掉


scaler = preprocessing.StandardScaler()#将Age和Fare的值压缩到（-1和1之间）
df['Age_scaled'] = scaler.fit_transform(df['Age'].values.reshape(-1,1))   
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1,1))

train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Embarked_.*|Sex_.*|Pclass_.*') 
train_np = train_df.as_matrix() 
# y即Survival结果 
y = train_np[:, 0] 
# X即特征属性值 
X = train_np[:, 1:] 

clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6) 
clf.fit(X, y)
scores = cross_val_score(clf, X, y, cv=10)#交叉验证
print(scores)
b=len(scores)
sum_score = 0
for i in scores:
    sum_score=sum_score+i
mean_score=sum_score/b
print(mean_score)

split_train, split_cv =train_test_split(df, test_size=0.3, random_state=0)
train_df = split_train.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Embarked_.*|Sex_.*|Pclass_.*')# 生成模型
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(train_df.as_matrix()[:,1:], train_df.as_matrix()[:,0])# 对cross validation数据进行预测
 
cv_df = split_cv.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Embarked_.*|Sex_.*|Pclass_.*')
predictions = clf.predict(cv_df.as_matrix()[:,1:])
 
origin_data_train = pd.read_csv("/Users/sunhao/Desktop/Train.csv")
bad_cases = origin_data_train.loc[origin_data_train['PassengerId'].isin(split_cv[predictions != cv_df.as_matrix()[:,0]]['PassengerId'].values)]
data_test = pd.read_csv("/Users/sunhao/Desktop/test.csv") 
data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0 # 接着我们对test_data做和train_data中一致的特征变换 
# 首先用同样的RandomForestRegressor模型填上丢失的年龄 
tmp_df = data_test[['Age','Fare', 'Parch', 'SibSp', 'Pclass']] 
null_age = tmp_df[data_test.Age.isnull()].as_matrix() # 根据特征属性X预测年龄并补上 
X = null_age[:, 1:] 
predictedAges = rfr.predict(X) 
data_test.loc[ (data_test.Age.isnull()), 'Age' ] = predictedAges 

dummies_Embarked = pd.get_dummies(data_test['Embarked'], prefix= 'Embarked') 
dummies_Sex = pd.get_dummies(data_test['Sex'], prefix= 'Sex') 
dummies_Pclass = pd.get_dummies(data_test['Pclass'], prefix= 'Pclass') 
df_test = pd.concat([data_test, dummies_Embarked, dummies_Sex, dummies_Pclass], axis=1) 
df_test.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Embarked'], axis=1, inplace=True) 
df_test['Age_scaled'] = scaler.fit_transform(df_test['Age'].values.reshape(-1,1)) 
df_test['Fare_scaled'] = scaler.fit_transform(df_test['Fare'].values.reshape(-1,1))
test = df_test.filter(regex='Age_.*|SibSp|Parch|Fare_.*|Embarked_.*|Sex_.*|Pclass_.*') 
predictions = clf.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].as_matrix(), 'Survived':predictions.astype(np.int32)}) 
result.to_csv("/Users/sunhao/Desktop/gender_submission.csv", index=False)